# SET UP ENVIRONMET

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'rail-demo-367913'

In [2]:
REGION = 'europe-west4'

In [3]:
from google.cloud import storage

import pandas as pd
import numpy as np

In [4]:
gcs = storage.Client(project = PROJECT_ID)

In [5]:
BUCKET = PROJECT_ID

---
## Create Storage Bucket
Check to see if bucket already exist and create if missing:
- [GCS Python Client](https://cloud.google.com/python/docs/reference/storage/latest/google.cloud.storage.client.Client)

In [6]:
if not gcs.lookup_bucket(BUCKET):
    bucketDef = gcs.bucket(BUCKET)
    bucket = gcs.create_bucket(bucketDef, project=PROJECT_ID, location=REGION)
    print(f'Created Bucket: {gcs.lookup_bucket(BUCKET).name}')
else:
    bucketDef = gcs.bucket(BUCKET)
    print(f'Bucket already exist: {bucketDef.name}')

Bucket already exist: rail-demo-367913


In [7]:
print(f'Review the storage bucket in the console here:\nhttps://console.cloud.google.com/storage/browser/{PROJECT_ID};tab=objects&project={PROJECT_ID}')

Review the storage bucket in the console here:
https://console.cloud.google.com/storage/browser/rail-demo-367913;tab=objects&project=rail-demo-367913


---
<a id = 'permissions'></a>
## Service Account & Permissions

This notebook instance is running as a service account in GCP.  This service account will also be used to run other services in Vertex AI like training jobs and pipelines.  The service account will need permission to interact with object in Cloud Storage which requires the role ([roles/storage.objectAdmin](https://cloud.google.com/storage/docs/access-control/iam-roles)).  

In [8]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'595378874286-compute@developer.gserviceaccount.com'

In [9]:
!gcloud services enable cloudresourcemanager.googleapis.com

In [10]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

ROLE
roles/owner
roles/storage.objectAdmin


In [11]:
!pip install kfp -U -q
!pip install google-cloud-pipeline-components -U -q

^C
ERROR: Operation cancelled by user


---
## Update AIPlatform Package:

The `google-cloud-aiplatform` package updates frequently.  Update it for latest functionality.

- [aiplatform Python Client](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform)
- [GitHub Repo for api-common-protos](https://github.com/googleapis/api-common-protos)

For a better understanding of the Vertex AI APIs client, version, and layers please review the tip here [aiplatform_notes.md](../Tips/aiplatform_notes.md).

In [12]:
!pip install googleapis-common-protos -U -q
!pip install google-cloud-aiplatform -U -q

# MODEL IMPLEMENTATION

In [13]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

gcs = storage.Client(project = PROJECT_ID)
BUCKET = PROJECT_ID

In [14]:
# source data
RAIL_PROJECT = PROJECT_ID
RAIL_DATASET = 'rail-demo-367913'

In [15]:
import tensorflow as tf
import numpy as np

In [16]:
list(bucketDef.list_blobs())

[<Blob: rail-demo-367913, crossval_saved_arrays/, 1670250560489619>,
 <Blob: rail-demo-367913, crossval_saved_arrays/0_images.npy, 1670250930482765>,
 <Blob: rail-demo-367913, crossval_saved_arrays/0_masks_mesh.npy, 1670251844968882>,
 <Blob: rail-demo-367913, crossval_saved_arrays/0_masks_wire.npy, 1670253001169768>,
 <Blob: rail-demo-367913, crossval_saved_arrays/1_images.npy, 1670250944933667>,
 <Blob: rail-demo-367913, crossval_saved_arrays/1_masks_mesh.npy, 1670252201340124>,
 <Blob: rail-demo-367913, crossval_saved_arrays/1_masks_wire.npy, 1670253575301645>,
 <Blob: rail-demo-367913, crossval_saved_arrays/2_images.npy, 1670250934782373>,
 <Blob: rail-demo-367913, crossval_saved_arrays/2_masks_mesh.npy, 1670252178103368>,
 <Blob: rail-demo-367913, crossval_saved_arrays/2_masks_wire.npy, 1670253843177141>,
 <Blob: rail-demo-367913, crossval_saved_arrays/3_images.npy, 1670251274325776>,
 <Blob: rail-demo-367913, crossval_saved_arrays/3_masks_mesh.npy, 1670252688450179>,
 <Blob: rail

In [18]:
"""#to download files from cloud storage
bucket_name = "rail-demo-367913"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

# When you have your files in a subfolder of the bucket.
my_prefix = "saved_arrays_multiclass/" # the name of the subfolder
blobs = bucket.list_blobs(prefix = my_prefix, delimiter = '/')

for blob in blobs:
    if(blob.name != my_prefix): # ignoring the subfolder itself 
        file_name = blob.name.replace(my_prefix, "")
        blob.download_to_filename(file_name) # download the file to the machine
        df = np.load(file_name) # load the data
        print(df)
"""

'#to download files from cloud storage\nbucket_name = "rail-demo-367913"\n\nstorage_client = storage.Client()\nbucket = storage_client.get_bucket(bucket_name)\n\n# When you have your files in a subfolder of the bucket.\nmy_prefix = "saved_arrays_multiclass/" # the name of the subfolder\nblobs = bucket.list_blobs(prefix = my_prefix, delimiter = \'/\')\n\nfor blob in blobs:\n    if(blob.name != my_prefix): # ignoring the subfolder itself \n        file_name = blob.name.replace(my_prefix, "")\n        blob.download_to_filename(file_name) # download the file to the machine\n        df = np.load(file_name) # load the data\n        print(df)\n'

In [19]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Conv2DTranspose, Concatenate
import random
from keras.callbacks import ModelCheckpoint
from keras import backend as K
import cv2
import os
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [20]:
DIM = 512
EPOCHS = 250
PATIENCE = 50
BATCH_SIZE = 8
OPTIMIZER = 'adam'
N_CHANNELS = 3
N_CLASSES = 3
DIM = 512
MODEL_NAME = "unet_multiclass.h5"

In [22]:

src_path = "saved_arrays_multiclass"
train_images = np.load(src_path + "/train_images.npy")[:10] #FIXME aggiungere tutte le immagini
train_masks = np.load(src_path + "/train_masks.npy")[:10]
val_images = np.load(src_path + "/val_images.npy")[:10]
val_masks = np.load(src_path + "/val_masks.npy")[:10]
test_images = np.load(src_path + "/test_images.npy")[:10]
test_masks = np.load(src_path + "/test_masks.npy")[:10]
print("train images", train_images.shape)
print("train masks", train_masks.shape)
print("val images", val_images.shape)
print("test images", test_images.shape)


train images (10, 512, 512, 3)
train masks (10, 512, 512, 3)
val images (10, 512, 512, 3)
test images (10, 512, 512, 3)


In [23]:

dst_dir = "samples_multiclass/during_training"
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

In [24]:

train_images = train_images/255.0
val_images = val_images/255.0
test_images = test_images/255.0


In [25]:

checkpoint_path = "checkpoints" #where to save the model checkpoints
if not os.path.exists(checkpoint_path):
    os.mkdir(checkpoint_path)



In [26]:
def mean_iou(y_true, y_pred, num_classes):
    correct_pred = np.zeros(num_classes) # int = (mesh : M, wire : W and background : B)
    den = np.zeros(num_classes) # den = M + W + B = (M or W or B) + (M and W and B) + ..

    for i in range (y_true.shape[0]):
        for j in range(y_true.shape[1]):
            for k in range(y_true.shape[2]):
                if (y_pred[i][j][k])==(y_true[i][j][k]):
                    correct_pred[(y_true[i][j][k])]+=1
                den[(y_pred[i][j][k])] += 1
                den[(y_true[i][j][k])] += 1
    mIoU = 0
    IoU_classes = []
    for i in range(num_classes):
        if den[i]!=0:
            IoU =correct_pred[i]/(den[i]-correct_pred[i])
            IoU_classes.append(IoU)

    mIoU=sum(IoU_classes)/len(IoU_classes)
    return mIoU

### UNET

In [50]:
!pip install -U segmentation-models


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 11.3 MB/s eta 0:00:00


In [51]:
%env SM_FRAMEWORK=tf.keras


env: SM_FRAMEWORK=tf.keras


In [52]:
import segmentation_models as sm
import tensorflow as tf
sm.set_framework('tf.keras')
sm.framework()
model = sm.Unet('inceptionv3',
                classes=3, 
                input_shape = (DIM,DIM,3), 
                activation='softmax', 
                encoder_freeze=False,
                )

Segmentation Models: using `tf.keras` framework.
87924736/87910968 [==============================] - 3s 0us/step


In [53]:
model.summary()


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d_100 (Conv2D)             (None, 256, 256, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_18 (BatchNo (None, 256, 256, 32) 96          conv2d_100[0][0]                 
__________________________________________________________________________________________________
activation_18 (Activation)      (None, 256, 256, 32) 0           batch_normalization_18[0][0]     
____________________________________________________________________________________________

In [48]:

def build_callbacks():
    checkpointer = [
          EarlyStopping(monitor="val_mean_iou", patience=PATIENCE, restore_best_weights=True, mode="max"),
          ModelCheckpoint(checkpoint_path + '/'+ MODEL_NAME,  monitor="val_mean_iou", save_best_only=True, mode='max') #best on iou
    ]
    return checkpointer

In [49]:

model = unet((DIM,DIM,3), N_CLASSES)
print(model.summary())

model.fit(train_images, train_masks,  batch_size=BATCH_SIZE, verbose=1, epochs=EPOCHS, validation_data=(val_images, val_masks), callbacks = build_callbacks())


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d_73 (Conv2D)              (None, 512, 512, 8)  224         input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_74 (Conv2D)              (None, 512, 512, 8)  584         conv2d_73[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_16 (MaxPooling2D) (None, 256, 256, 8)  0           conv2d_74[0][0]                  
____________________________________________________________________________________________

TypeError: in user code:

    /opt/conda/lib/python3.7/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)

    TypeError: tf__mean_iou() missing 1 required positional argument: 'num_classes'


In [54]:
len(model.layers)


352

In [57]:
from segmentation_models import metrics
model.compile(optimizer = tf.keras.optimizers.Adam(0.00005,decay=1e-6),
              loss=sm.losses.DiceLoss(),
              metrics = [sm.metrics.f1_score,
                         sm.metrics.iou_score]
)

In [58]:
model_history = model.fit(train_images, train_masks,
                          batch_size = BATCH_SIZE,
                          epochs=EPOCHS,
                          validation_data=(val_images, val_masks),
                          shuffle=False)

Epoch 1/250


TypeError: in user code:

    /opt/conda/lib/python3.7/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.7/site-packages/segmentation_models/losses.py:97 __call__  *
        pr,
    /opt/conda/lib/python3.7/site-packages/segmentation_models/base/functional.py:145 f_score  *
        tp = backend.sum(gt * pr, axis=axes)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/math_ops.py:1383 binary_op_wrapper
        raise e
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/math_ops.py:1367 binary_op_wrapper
        return func(x, y, name=name)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/math_ops.py:1710 _mul_dispatch
        return multiply(x, y, name=name)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:206 wrapper
        return target(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/math_ops.py:5

In [35]:


history = model.fit(train_images, train_masks, 
                    batch_size = BATCH_SIZE, 
                    verbose=1, 
                    epochs=EPOCHS, 
                    validation_data=(val_images, val_masks), 
                    shuffle=False)

                    #Save the model for future use

Epoch 1/250


TypeError: 'NoneType' object is not callable

In [44]:

def build_callbacks():
    checkpointer = [
          EarlyStopping(monitor="val_mean_iou", patience=PATIENCE, restore_best_weights=True, mode="max"),
          ModelCheckpoint(checkpoint_path + '/'+ MODEL_NAME,  monitor="val_mean_iou", save_best_only=True, mode='max') #best on iou
    ]
    return checkpointer

In [45]:

model = unet(sz=(DIM,DIM,3))
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d_27 (Conv2D)              (None, 512, 512, 8)  224         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_28 (Conv2D)              (None, 512, 512, 8)  584         conv2d_27[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_6 (MaxPooling2D)  (None, 256, 256, 8)  0           conv2d_28[0][0]                  
____________________________________________________________________________________________

In [46]:

model.fit(train_images, train_masks,  batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(val_images, val_masks), callbacks = build_callbacks())


Epoch 1/250
108/108 [==============================] - 16s 121ms/step - loss: nan - accuracy: 0.4303 - mean_iou: 0.0137 - val_loss: nan - val_accuracy: 0.2980 - val_mean_iou: 0.0000e+00
Epoch 2/250
108/108 [==============================] - 10s 92ms/step - loss: nan - accuracy: 0.3371 - mean_iou: 0.0000e+00 - val_loss: nan - val_accuracy: 0.2980 - val_mean_iou: 0.0000e+00
Epoch 3/250
108/108 [==============================] - 10s 92ms/step - loss: nan - accuracy: 0.3371 - mean_iou: 0.0000e+00 - val_loss: nan - val_accuracy: 0.2980 - val_mean_iou: 0.0000e+00
Epoch 4/250
108/108 [==============================] - 10s 93ms/step - loss: nan - accuracy: 0.3371 - mean_iou: 0.0000e+00 - val_loss: nan - val_accuracy: 0.2980 - val_mean_iou: 0.0000e+00
Epoch 5/250
108/108 [==============================] - 10s 93ms/step - loss: nan - accuracy: 0.3371 - mean_iou: 0.0000e+00 - val_loss: nan - val_accuracy: 0.2980 - val_mean_iou: 0.0000e+00
Epoch 6/250
108/108 [==============================] - 10s

KeyboardInterrupt: 